# Uitwerkingen opgaven data-analyse

## Opgave 1.3

We maken hier gebruik van de error function, gedefinieerd in `scipy.special.erf`. Er is ook een complementaire error function, gedefinieerd als $\mathrm{erfc} = 1 - \mathrm{erf}$. Vergeet niet de factor $\sqrt{2}$, aangezien $\mathrm{erf}$ een vereenvoudigde functie is, en het argument nog omgerekend moet worden.

De waarschijnlijkheid dat een waarde méér dan $1.23\sigma$ van het gemiddelde afligt is gegeven door:

In [ ]:
from math import sqrt
from scipy.special import erf, erfc

In [ ]:
erfc(1.23 / sqrt(2))

De waarschijnlijkheid dat een waarde méér dan $2.43\sigma$ *boven* het gemiddelde ligt is gegeven door:

In [ ]:
erfc(2.43 / sqrt(2)) / 2

De waarschijnlijkheid dat een waarde meer dan $0.5\sigma$, maar minder dan $1.5\sigma$ van het gemiddelde verwijderd is, wordt gegeven door:

In [ ]:
erf(1.5 / sqrt(2)) - erf(.5 / sqrt(2))

We willen weten binnen hoeveel standaarddeviaties van het gemiddelde de waarschijnlijkheid dat een waarde gevonden wordt gelijk is aan $50\,\%$. Gebruik hiervoor de inverse error function `erfinv`. Vergeet de factor $\sqrt{2}$ niet!

In [ ]:
from scipy.special import erfinv

In [ ]:
erfinv(.5) * sqrt(2)

## Opgave 1.4

We maken gebruik van pandas, een python data-analyse pakket. We importeren de datafile:

In [ ]:
import pandas as pd

In [ ]:
data = pd.read_csv('10-metingen.txt')

In [ ]:
print(data)

...en bekijken wat statistieken:

In [ ]:
data.describe()

De standaarddeviatie van het gemiddelde is:

In [ ]:
x = data.x
x.std() / sqrt(x.count())

Het resultaat van ons experiment is dus $x = 71.8 \pm 1.7$ of $x = 71.8(17)$.

Voor De kans op een volgende meting met $x\geq 75$ berekenen we eerst hoeveel standaarddeviaties de meting verwijderd is van het gemiddelde. Vervolgens berekenen we de kans op een meting hóger dan dat aantal standaarddeviaties:

In [ ]:
dist = 75 - x.mean()
dist_sigma = dist / x.std()
erfc(dist_sigma / sqrt(2)) / 2

## Opgave 1.5

We gaan plotten, en we moeten `matplotlib` vertellen dat we de plots in de notebook willen zien:

In [ ]:
%matplotlib inline

We openen de datafile en maken een histogram van $y$:

In [ ]:
data = pd.read_csv('80-metingen.txt')
data.describe()

In [ ]:
data.y.hist()

Het is best interessant dat dit histogram er heel anders uitziet dan het antwoord uit de uitwerkingen. Dat histogram is gemaakt met Origin, en dat lijkt standaard iets beter op zoek te gaan naar aardige bingrenzen. Niet simpelweg het minimum en het maximum nemen en opdelen, maar afronden op mooie, ronde getallen. Handmatig kan dat in Python. Voor stapgroottes kun je het beste `arange` gebruiken. Let er daarbij wel op dat `arange` niet inclusief is. Het maximum wordt niet meegenomen. Als je dat ietsje groter maakt dan de bingrens, dus 101 i.p.v. 100, dan komt het goed:

In [ ]:
import numpy as np
bins = np.arange(20, 101, 10)
data.y.hist(bins=bins)

### Methode A: fitten aan volledige dataset

Het fitten van een bekende distributie (zie `scipy.stats`) aan de data gaat vrij eenvoudig. Daar heb je niet eens een histogram voor nodig:

In [ ]:
from scipy import stats
mean, sigma = stats.norm.fit(data.y)
mean, sigma

De meest waarschijnlijke distributie heeft een gemiddelde van 61.6 en een standaarddeviatie van 16.7. Je krijgt alleen de distributie, genormeerd, dus als je dat samen met het histogram wilt plotten moet je eerst nog schalen met een factor $N^2 / N_\mathrm{bins}$:

In [ ]:
import matplotlib.pyplot as plt

data.y.hist(bins=bins)

scale = data.y.count() ** 2 / len(bins - 1)
x = np.linspace(20, 100, 50)
plt.plot(x, scale * stats.norm.pdf(x, mean, sigma))

Het is dan wellicht interessanter om het histogram weer te geven als waardes met foutenvlaggen. Eerst wil je de waardes van het histogram, zonder te plotten. Dan moet je de middens van de bins berekenen, en dat plotten met de fouten $\sqrt{N}$:

In [ ]:
n, _ = np.histogram(data.y, bins=bins)
xbins = (bins[:-1] + bins[1:]) / 2
plt.errorbar(xbins, n, yerr=np.sqrt(n), fmt='o')

plt.plot(x, scale * stats.norm.pdf(x, mean, sigma))

Merk op dat de waardes van de fit iets verschillen van die van Origin, helaas. De volledige code om het laatste plaatje te maken wordt dan:

In [ ]:
# imports
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

# read datafile
data = pd.read_csv('80-metingen.txt')

# calculate histogram
bins = np.arange(20, 101, 10)
n, _ = np.histogram(data.y, bins=bins)

# plot histogram with bin centers and error bars
xbins = (bins[:-1] + bins[1:]) / 2
plt.errorbar(xbins, n, yerr=np.sqrt(n), fmt='o')

# fit normal distribution to data
mean, sigma = stats.norm.fit(data.y)

# plot the scaled distribution
scale = data.y.count() ** 2 / len(bins - 1)
x = np.linspace(20, 100, 50)
plt.plot(x, scale * stats.norm.pdf(x, mean, sigma))

We vertrouwen de data wel. De fit gaat door of vlak langs alle foutenvlaggen.

Dit is wel meer werk dan in Origin, vooral als je nog niet zo handig bent. Zeker ook meer om te onthouden. **Merk op dat de fit gedaan is op de volledige set waarnemingen, dus niet op het histogram. Als er gefit moet worden aan een histogram, of als je een zelf-gedefinieerde functie wilt fitten, dan moet je overstappen naar least-squares fitting.**

### Methode B: fitten aan een histogram

Als je least-squares wilt fitten, gebruik dan de `lmfit` bibliotheek. Die is handiger dan `scipy` gebruiken.

In [ ]:
from lmfit import models

Je kunt veel verschillende modellen gebruiken, b.v. een `GaussianModel`. De werkwijze is als volgt: éérst initialiseer je het model en vervolgens voer je de fit uit. Het is handig om voor de fit een *first guess* op te geven, een afschatting van de parameters. Je kunt dit handmatig doen, maar wij gebruiken hier de method `guess` van het model. Dit geeft een afschatting van de parameters...

In [ ]:
gauss = models.GaussianModel()
first_guess = gauss.guess(n, x=xbins)
first_guess.pretty_print()

...die je vervolgens in de fit kunt stoppen. Denk hierbij aan de fout op de datapunten. In de fit kun je een *gewicht* toekennen, dat omgekeerd evenredig is met de fout.

In [ ]:
fit = gauss.fit(n, x=xbins, weights=1 / np.sqrt(n), params=first_guess)
print(fit.fit_report())

In [ ]:
fit.plot(numpoints=50)

### Vergelijking van de methodes A en B

Je kunt je nu afvragen welke methode beter is. Bovenstaande fit ziet er prachtig uit. Toch is er nog een subtiliteit: het histogram hangt nogal af van de keuze voor de bins. In bovenstaand voorbeeld hadden we de bins zelf gekozen. Een andere keuze voor de bins, levert een andere fit op:

In [ ]:
n, bins = np.histogram(data.y)
xbins = (bins[:-1] + bins[1:]) / 2

first_guess = gauss.guess(n, x=xbins)
fit = gauss.fit(n, x=xbins, weights=1 / np.sqrt(n), params=first_guess)
print(fit.fit_report())
fit.plot(numpoints=50)

Deze fit heeft een iets ander centrum, een andere breedte, en een fors lagere amplitude.

## Opgave 1.6

We importeren de dataset en kijken even of dat goed ging.

In [ ]:
data = pd.read_csv('verval.txt')

In [ ]:
data.describe()

We zien een kolom **tijd** en een kolom **counts**, met negen meetpunten. De fouten op de meetpunten stonden niet in het bestand, maar wel in de opgave. De fout op het aantal counts $N$ wordt gegeven door $\sqrt N$. We maken een nieuwe kolom `yerr` als volgt:

In [ ]:
data['yerr'] = np.sqrt(data.counts)

Merk op dat we de data nu kunnen bekijken door zowel `data['yerr']` als `data.yerr` te typen, maar dat we nieuwe kolommen alléén kunnen maken met de `data['yerr']`-notatie. We inspecteren even de eerste vijf regels om te zien of alles goed ging:

In [ ]:
data.head()

We maken een plot van counts tegen tijd, met de juiste foutvlaggen:

In [ ]:
data.plot.scatter('tijd', 'counts', yerr='yerr')

De vraag is nu hoe we het beste om kunnen gaan met de achtergrond. Eenvoudig negeren, fitten aan een exponentiële functie plus een constante achtergrond of de achtergrond éérst aftrekken van de data? We bekijken hieronder de drie methodes.

### Methode A: negeren van de achtergrond

In [ ]:
model = models.ExponentialModel()
fit = model.fit(data.counts, x=data.tijd, weights=1 / data.yerr)
print(fit.fit_report())
fit.plot(numpoints=50)

Op het oog ziet dit er prima uit, maar $\chi_\mathrm{red}^2 = 1.659$, vrij groot. Aan de residuals is nog niet heel veel te zien. Je moet langer doormeten om duidelijk te kunnen zien dat de exponentiële functie naar nul gaat, terwijl de metingen dat niet doen.

### Methode B: fitten aan exponentiële functie plus constante achtergrond

We kunnen met `lmfit` eenvoudig modellen optellen:

In [ ]:
model = models.ExponentialModel() + models.ConstantModel()

Vervolgens fitten we dit model aan de data en bekijken het resultaat:

In [ ]:
fit = model.fit(data.counts, x=data.tijd, weights=1 / data.yerr)
print(fit.fit_report())
fit.plot(numpoints=50)

Dat is een mooi resultaat. Op het oog gaat de fit goed door de punten en $\chi^2_\mathrm{red} = 1.167$. Dit resultaat is waarschijnlijker dan het resultaat van methode A, waar we de achtergrond negeren.

### Methode C: Achtergrond van de data aftrekken

Uit de opgave weten we dat er wel degelijk een achtergrond is, en dat die bepaald is op gemiddeld 12 counts per minuut. We kunnen een nieuwe kolom maken:

In [ ]:
data['nobkg'] = data.counts - 12

De vraag is nu: wat doen we met de fout op deze data? Gebruiken we dezelfde fout als eerst? Maar dat was de wortel van het aantal counts mét achtergrond. De wortel van het aantal counts zónder achtergrond, dus `sqrt(data.nobkg)`, ligt voor de hand maar is niet juist. Niet elke meting had immers precies 12 counts aan achtergrond. We trekken 12 counts af, maar de verwachte onzekerheid op het *meten* van de achtergrond is $\sqrt{12}$. Dat telt mee.

We moeten de regels voor de foutenberekening aanroepen: $N_\mathrm{nobkg} = N - N_\mathrm{bkg}$. Dus de onzekerheid wordt gegeven door $\delta N_\mathrm{nobkg} = \sqrt{(\delta N)^2 + (\delta N_\mathrm{bkg})^2}$. De onzekerheid wordt, na aftrekken van de achtergrond, dus eigenlijk zelfs *groter*. Nu was de achtergrond gelukkig heel nauwkeurig bepaald, dus de onzekerheid op de grootte van de achtergrond is verwaarloosbaar klein. Dat betekent dat $\delta N_\mathrm{nobkg} = \delta N = \sqrt{N}$. We houden dus de oorspronkelijke onzekerheid.

In [ ]:
model = models.ExponentialModel()
fit = model.fit(data.nobkg, x=data.tijd, weights=1 / data.yerr)
print(fit.fit_report())
fit.plot(numpoints=50)

De $\chi_\mathrm{red}^2 = 1.005$. Dit is dus de meest nauwkeurige methode om de halfwaardetijd te bepalen. De halfwaardetijd wordt gegeven door $t_{1/2} = \lambda\ln 2$.

In [ ]:
fit.params['decay'].value * np.log(2)

Dus $t_{1/2} = 1.196$ s. Het kan makkelijker zijn om een eigen model te maken, waar $t_{1/2}$ expliciet in voorkomt: $N(t) = N_0 * \frac{1}{2}^{t/t_{1/2}}$.

In [ ]:
verval = models.Model(lambda t, N0, thalf: N0 * .5 ** (t / thalf))

Meestal moeten we een schatting opgeven van de parameters. Zonder die schatting is het goed mogelijk dat het fit-algoritme zoekt bij zulke onrealistische waardes dat er 'oneindig' uit de functie komt. Daarn kan het algoritme niet goed mee omgaan. We kiezen gewoon de waarde 1 voor beide parameters:

In [ ]:
guess = verval.make_params(N0=1, thalf=1)

In [ ]:
fit = verval.fit(data.nobkg, t=data.tijd, weights=1 / data.yerr, params=guess)
print(fit.fit_report())
fit.plot(numpoints=50)

En we lezen nu meteen af dat $t_{1/2} = 1.196 \pm 0.027$ s, ofwel $t_{1/2} = 1.196(27)$ s.